In [3]:
import os
from llmner import ZeroShotNer, FewShotNer
from datasets import load_dataset
from seqeval.metrics import classification_report
from llmner.data import PromptTemplate
import json
import numpy as np

# We change api base for deepinfra 
os.environ["OPENAI_API_BASE"] = "https://api.deepinfra.com/v1/openai"
os.environ["OPENAI_API_KEY"] = "<Your Deep Infra API key>"

In [4]:
# Test with small data

conll2003 = load_dataset("conll2003", split="test[:5%]")
conll2002 = load_dataset("conll2002", "es", split="test[:5%]")

In [5]:
# Mapping from Number to CoNLL-2003 Tag
n_to_conll = {0:'O', 1:'B-PER', 2:'I-PER', 3:'B-ORG', 4:'I-ORG', 5:'B-LOC', 6:'I-LOC',7: 'B-MISC',8: 'I-MISC' }
entity_set = ["PER", "ORG", "LOC", "MISC"]

In [6]:
# Formatting annotations

conll2003_annotations_conll = []
conll2002_annotations_conll = []

for i in range(len(conll2003)):
    tokens = conll2003[i]["tokens"]
    conll2003_annotations_conll.append([(tokens[j] , n_to_conll[conll2003[i]["ner_tags"][j]]) for j in range(len(tokens))])


for i in range(len(conll2002)):
    tokens = conll2002[i]["tokens"]
    conll2002_annotations_conll.append([(tokens[j] , n_to_conll[conll2002[i]["ner_tags"][j]]) for j in range(len(tokens))])

conll2003_annotations_seqeval = [ [annotation[j][1] for j in range(len(annotation))] for annotation in conll2003_annotations_conll]
conll2002_annotations_seqeval = [ [annotation[j][1] for j in range(len(annotation))] for annotation in conll2002_annotations_conll]

In [7]:
from llmner.data import PromptTemplate

template_es = PromptTemplate(
    inline_single_turn="""Eres un reconocedor de entidades nombradas que debe detectar las siguientes entidades:
    {entities}
    Debes responder con el mismo texto de entrada, pero con las entidades nombradas anotadas con anotaciones de etiquetas en línea (<entity>texto</entity>), donde cada etiqueta corresponde a un nombre de entidad, por ejemplo: <name>John Doe</name> es el propietario de <organization>ACME</organization>.
    Las únicas etiquetas disponibles son: {entity_list}, no puedes agregar más etiquetas que las incluidas en esa lista.
    IMPORTANTE: NO DEBE CAMBIAR EL TEXTO DE ENTRADA, SOLO AGREGAR LAS ETIQUETAS.""",
    inline_multi_turn_default_delimiters="""Eres un reconocedor de entidades nombradas que debe detectar las siguientes entidades:
    {entities}
    Debes responder con el mismo texto de entrada, pero con una sola entidad anotada con anotaciones de etiquetas en línea (<entity>texto</entity>), donde la etiqueta corresponde a un nombre de entidad, por ejemplo, primero te pido que anotes los nombres: <name>John Doe</name> es el propietario de ACME y luego te pido que anotes las organizaciones: John Doe es el propietario de <organization>ACME</organization>.
    Las únicas etiquetas disponibles son: {entity_list}, no puedes agregar más etiquetas que las incluidas en esa lista.
    IMPORTANTE: NO DEBE CAMBIAR EL TEXTO DE ENTRADA, SOLO AGREGAR LAS ETIQUETAS""",
    inline_multi_turn_custom_delimiters="""Eres un reconocedor de entidades nombradas que debe detectar las siguientes entidades:
    {entities}
    Debes responder con el mismo texto de entrada, pero con una sola entidad anotada con anotaciones de etiquetas en línea ({start_token}texto{end_token}), donde la etiqueta corresponde a un nombre de entidad, por ejemplo, primero te pido que anotes los nombres: {start_token}Jhon Doe{end_token} es el propietario de ACME y luego te pido que anotes las organizaciones: John Doe es el propietario de {start_token}ACME{end_token}.
    Las únicas etiquetas disponibles son: {entity_list}, no puedes agregar más etiquetas que las incluidas en esa lista.
    IMPORTANTE: NO DEBE CAMBIAR EL TEXTO DE ENTRADA, SOLO AGREGAR LAS ETIQUETAS""",
    json_single_turn="""Eres un reconocedor de entidades nombradas que debe detectar las siguientes entidades:
    {entities}
    Debes responder con formato JSON, donde cada clave corresponde a una clase de entidad, y el valor es una lista de las menciones de la entidad, por ejemplo: {{"name": ["John Doe"], "organization": ["ACME"]}}.
    Las únicas etiquetas disponibles son: {entity_list}, no puedes agregar más etiquetas que las incluidas en esa lista.
    IMPORTANTE: SU SALIDA DEBE SER SOLO UN JSON EN EL FORMATO {{"entity_class": ["entity_mention_1", "entity_mention_2"]}}. NO SE PERMITE OTRO FORMATO.""",
    json_multi_turn="""Eres un reconocedor de entidades nombradas que debe detectar las siguientes entidades:
    {entities}
    Debes responder con el mismo texto de entrada, pero con una sola entidad anotada con formato JSON, donde la clave corresponde a una clase de entidad, por ejemplo, primero te pido que anotes los nombres: {{"name": ["John Doe"]}} y luego te pido que anotes las organizaciones: {{"organization": ["ACME"]}}
    Las únicas etiquetas disponibles son: {entity_list}, no puedes agregar más etiquetas que las incluidas en esa lista.
    IMPORTANTE: SU SALIDA DEBE SER SOLO UN JSON EN EL FORMATO {{"entity_class": ["entity_mention_1", "entity_mention_2"]}}. NO SE PERMITE OTRO FORMATO.""",
    multi_turn_prefix="""En el siguiente texto, anota la entidad """,
    pos="""Eres un etiquetador de partes del discurso que debe detectar las etiquetas de partes del discurso. Responda con el mismo texto de entrada, pero con las etiquetas de partes del discurso después de cada palabra, por ejemplo: John/NNP Doe/NNP es/VBZ el/DT propietario/NN de/IN ACME/NNP.""",
    pos_answer_prefix="""Este es el texto con las etiquetas de partes del discurso:""",
    final_message_prefix = """Ahora, anota el siguiente documento con todas las entidades ({entity_list}):"""
)

entities_en = {
    "LOC": "roads, trajectories, regions, structures, natural locations, public places, commercial places, assorted buildings, abstract places (e.g. the free world)",
    "PER": "first, middle and last names of people, animals and fictional characters aliases",
    "ORG": "companies, subdivisions of companies, brands, political movements, government bodies, publications, musical companies, public organisations, other collections of people",
    "MISC": "words of which one part is a location, organisation, miscellaneous, or person, adjectives and other words derived from a word which is location, organisation, miscellaneous, or person, religions, political ideologies, nationalities, languages, programs, events, wars, sports related names, titles, slogans, eras in time types of objects",
}
entities_es = {
    "LOC": "carreteras, trayectorias, regiones, estructuras, lugares naturales, lugares públicos, lugares comerciales, edificios varios, lugares abstractos (por ejemplo, el mundo libre)",
    "PER": "nombres de personas, animales y personajes de ficción, alias",
    "ORG": "empresas, subdivisiones de empresas, marcas, movimientos políticos, organismos gubernamentales, publicaciones, empresas musicales, organizaciones públicas, otras colecciones de personas",
    "MISC": "palabras de las cuales una parte es una ubicación, organización, miscelánea o persona, adjetivos y otras palabras derivadas de una palabra que es una ubicación, organización, miscelánea o persona, religiones, ideologías políticas, nacionalidades, idiomas, programas, eventos, guerras, nombres relacionados con los deportes, títulos, eslóganes, épocas en tipos de objetos de tiempo",
}

In [8]:
# Quit annotation with different length
def get_different_length_annotations(annotations, predictions):
    annotation_filtered = []
    prediction_filtered = []
    for i in range(len(annotations)):
        if len(annotations[i]) == len(predictions[i]):
            annotation_filtered.append(annotations[i])
            prediction_filtered.append(predictions[i])
    return annotation_filtered, prediction_filtered, abs(len(annotations) - len(annotation_filtered))

In [9]:
zero_shot_json_multi_turn_model_2003 = ZeroShotNer(
    temperature=0.0,
    answer_shape="json",
    prompting_method="multi_turn",
    multi_turn_delimiters=None,
    augment_with_pos=False,
    model="meta-llama/Llama-2-70b-chat-hf",
    final_message_with_all_entities=True
)
zero_shot_json_multi_turn_model_2003.contextualize(entities=entities_en)

zero_shot_json_multi_turn_model_2003_pred = zero_shot_json_multi_turn_model_2003.predict_tokenized(conll2003["tokens"], max_workers=-1)

  0%|          | 0/173 [00:00<?, ? example/s]

Found 0 matches for Al-Ain in AL-AIN, United Arab Emirates 1996-12-06.
  9%|▊         | 15/173 [00:33<03:26,  1.31s/ example]Found 0 matches for Rome in ROME 1996-12-06.
Found 0 matches for Italy in RUGBY UNION - CUTTITTA BACK FOR ITALY AFTER A YEAR..
Found 0 matches for Rome in ROME 1996-12-06.
Found 0 matches for Main Street in '.
Found 0 matches for New York City in '.
Found 0 matches for Times Square in '.
Found 0 matches for John Smith in '.
Found 0 matches for Apple in '.
 10%|▉         | 17/173 [00:52<09:24,  3.62s/ example]Found 0 matches for Now in '.
Found 0 matches for annotate in '.
 16%|█▌        | 27/173 [01:07<04:38,  1.91s/ example]Found 0 matches for Japan in SOCCER - LATE GOALS GIVE JAPAN WIN OVER SYRIA..
Found 0 matches for Syria in SOCCER - LATE GOALS GIVE JAPAN WIN OVER SYRIA..
Found 0 matches for Al-Ain in AL-AIN, United Arab Emirates 1996-12-06.
Found 0 matches for Japan in SOCCER - LATE GOALS GIVE JAPAN WIN OVER SYRIA..
Found 0 matches for Syria in SOCCER - LATE

In [10]:
zero_shot_json_multi_turn_model_2002 = ZeroShotNer(
    temperature=0.0,
    answer_shape="json",
    prompting_method="multi_turn",
    multi_turn_delimiters=None,
    augment_with_pos=False,
    model="meta-llama/Llama-2-70b-chat-hf",
    prompt_template=template_es,
    final_message_with_all_entities=True
)
zero_shot_json_multi_turn_model_2002.contextualize(entities=entities_es)

zero_shot_json_multi_turn_model_2002_pred = zero_shot_json_multi_turn_model_2002.predict_tokenized(conll2002["tokens"], max_workers=-1)

  0%|          | 0/76 [00:00<?, ? example/s]Found 0 matches for Arévalo (Avila) in Arévalo (Avila), 23 may (EFE)..
Found 0 matches for España in García Aranda presentó a la prensa el sistema Amadeus, que utilizan la mayor parte de las agencias de viajes españolas para reservar billetes de avión o tren, así como plazas de hotel, y que ahora pueden utilizar también los usuarios finales a través de Internet..
Found 0 matches for New York City in -.
Found 0 matches for New York City in -.
Found 0 matches for John Doe in -.
Found 0 matches for John Doe in -.
Found 0 matches for ACME in -.
Found 0 matches for ACME in -.
Found 0 matches for familia gitana in La demora fue aprovechada por una familia de la comunidad gitana que reside en la localidad para ocupar una de las viviendas vacías, lo que originó un nuevo conflicto social después de los problemas de convivencia surgidos hace unas semanas entre los vecinos, que llegaron a exigir el destierro de varios jóvenes conflictivos..
  1%|▏      

In [11]:
zero_shot_json_multi_turn_model_2003_pred = [ [annotation[j][1] for j in range(len(annotation))] for annotation in zero_shot_json_multi_turn_model_2003_pred]
zero_shot_json_multi_turn_model_2002_pred = [ [annotation[j][1] for j in range(len(annotation))] for annotation in zero_shot_json_multi_turn_model_2002_pred]

print("CoNLL 2003:") 
comparation_conll2003 , comparation_pred, delta = get_different_length_annotations(conll2003_annotations_seqeval, zero_shot_json_multi_turn_model_2003_pred)
print(classification_report(comparation_conll2003, comparation_pred))
print(f"{delta} annotations have different length")
print("---------------------------------------------")


print("CoNLL 2002:")
comparation_conll2002 , comparation_pred, delta = get_different_length_annotations(conll2002_annotations_seqeval, zero_shot_json_multi_turn_model_2002_pred)
print(classification_report(comparation_conll2002, comparation_pred))
print(f"{delta} annotations have different length")

CoNLL 2003:
              precision    recall  f1-score   support

         LOC       0.41      0.54      0.47       104
        MISC       0.02      0.16      0.04        31
         ORG       0.22      0.40      0.28        30
         PER       0.79      0.68      0.73       180

   micro avg       0.35      0.57      0.44       345
   macro avg       0.36      0.45      0.38       345
weighted avg       0.56      0.57      0.55       345

0 annotations have different length
---------------------------------------------
CoNLL 2002:
              precision    recall  f1-score   support

         LOC       0.48      0.45      0.47        53
        MISC       0.05      0.41      0.08        17
         ORG       0.71      0.48      0.58        91
         PER       0.53      0.72      0.61        40

   micro avg       0.33      0.52      0.40       201
   macro avg       0.44      0.52      0.43       201
weighted avg       0.56      0.52      0.51       201

2 annotations have diffe